# Different model check

In [3]:
from transformers import pipeline

sentiment_analysis = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

def analyze_sentiment(text):
    results = sentiment_analysis(text)
    sentiment_results = {result['label']: result['score'] for result in results}
    return sentiment_results

analyze_sentiment("I am not feeling well about my research project")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'disappointment': 0.44469761848449707}

In [ ]:
def categorize_emotion(sentiment_results):
    stress_emotions = ['anger', 'fear', 'anxiety']
    frustration_emotions = ['annoyance', 'disappointment']
    
    stress_score = sum(sentiment_results.get(emotion, 0) for emotion in stress_emotions)
    frustration_score = sum(sentiment_results.get(emotion, 0) for emotion in frustration_emotions)
    
    if stress_score > 0.5:
        return "Stressed"
    elif frustration_score > 0.5:
        return "Frustrated"
    else:
        return "Neutral"
    
    

In [5]:
def real_time_sentiment_analysis(user_input):
    sentiment_results = analyze_sentiment(user_input)
    emotion_category = categorize_emotion(sentiment_results)
    return emotion_category

In [6]:
def refine_response(emotion_category, time_of_day):
    if emotion_category in ["Stressed", "Frustrated"]:
        return "emotionally_supportive"
    elif emotion_category == "Neutral":
        if time_of_day == "evening":
            return "informative"
        else:
            return "standard"

In [7]:
user_input = "I'm feeling overwhelmed with all this work!"
emotion = real_time_sentiment_analysis(user_input)
response_type = refine_response(emotion, "evening")

print(f"Detected emotion: {emotion}")
print(f"Response type: {response_type}")

Detected emotion: Neutral
Response type: informative


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Okay I understand, Would it help him"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) neutral 0.6971
2) positive 0.2617
3) negative 0.0412


# Transformer from huggingface for sentiment

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ValenHumano/roberta-base-bne-detector-de-stress")
model = AutoModelForSequenceClassification.from_pretrained("ValenHumano/roberta-base-bne-detector-de-stress")

model

In [9]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = ["I am not having a great day"]

model_outputs = classifier(sentences)
print(model_outputs[0])
# produces a list of dicts for each of the labels


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'disappointment', 'score': 0.4666949212551117}, {'label': 'sadness', 'score': 0.39849498867988586}, {'label': 'annoyance', 'score': 0.06806596368551254}, {'label': 'neutral', 'score': 0.05703023821115494}, {'label': 'disapproval', 'score': 0.04423929750919342}, {'label': 'nervousness', 'score': 0.014850744046270847}, {'label': 'realization', 'score': 0.014059885405004025}, {'label': 'approval', 'score': 0.011267459020018578}, {'label': 'joy', 'score': 0.006303390488028526}, {'label': 'remorse', 'score': 0.006221481133252382}, {'label': 'caring', 'score': 0.006029403302818537}, {'label': 'embarrassment', 'score': 0.005265478044748306}, {'label': 'anger', 'score': 0.004981435369700193}, {'label': 'disgust', 'score': 0.004259033594280481}, {'label': 'grief', 'score': 0.004002134781330824}, {'label': 'confusion', 'score': 0.0033829149324446917}, {'label': 'relief', 'score': 0.0031404958572238684}, {'label': 'desire', 'score': 0.0028274699579924345}, {'label': 'admiration', 'scor